In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
====================================================================================
📘 README: NSE F&O SENTIMENT ANALYZER (No-Argument Version)
====================================================================================

This script computes **derivatives-based sentiment** for NSE equities 
using Futures and Options data from NSE’s public API.

⚙️ HOW TO USE
-------------
1. Edit the `SYMBOLS` list near the top of this file.
   Example:
       SYMBOLS = ["RELIANCE", "HDFCBANK", "INFY", "ABB"]
2. Install dependencies:
       pip install requests pandas numpy pytz python-dateutil
3. Run:
       python fno_sentiment.py
4. Check results:
       outputs/YYYY-MM-DD/fno_sentiment_summary.csv
       outputs/YYYY-MM-DD/fno_sentiment_details.csv

------------------------------------------------------------------------------------
📊 HOW SENTIMENT IS CALCULATED
------------------------------------------------------------------------------------

🔹 OPTIONS COMPONENTS
----------------------
1. **Put–Call Ratio (PCR)**  
   PCR = total Put OI / total Call OI  
   Normalized: `pcr_score = tanh((PCR − 1.0) / 0.4)`  
   - PCR > 1.3 → bullish tilt  
   - PCR < 0.7 → bearish tilt  

2. **ΔOI Bias (doi_net)**  
   doi_net = (Σ ΔOI_puts) − (Σ ΔOI_calls)  
   Normalized: `oi_bias_score = tanh(doi_net / max(|doi_net|, 1))`  
   Positive = bullish (more put build-up)

3. **ATM IV & IV Skew**  
   - Take ±2 strikes around the ATM by index  
   - Compute avg CE_IV, avg PE_IV → `iv_skew = PE_IV − CE_IV`  
   - Convert to score: `iv_trend_score = tanh(iv_skew / max(|iv_skew|, 5))`  
   Positive skew ⇒ market paying more for puts ⇒ mild bullish sentiment.

🔹 FUTURES COMPONENTS
----------------------
1. **Basis** = Near Future LTP − Spot  
   **Basis% annualized** = (basis/spot) × (365 / days_to_expiry) × 100  
   - Positive basis → bullish carry  
   - Negative basis → bearish discount  

2. **Roll Spread** = Next Future − Near Future  
   Positive widening → bullish rollover  

3. **Regime (Price vs ΔOI):**  
   | ΔPrice | ΔOI | Interpretation | Score |
   |---------|------|----------------|--------|
   | ↑ | ↑ | Long Buildup | +1.0 |
   | ↓ | ↑ | Short Buildup | −1.0 |
   | ↓ | ↓ | Long Unwinding | −0.4 |
   | ↑ | ↓ | Short Covering | +0.4 |

🔹 COMPOSITE SENTIMENT SCORE
-----------------------------
Weighted combination:
    Score = 0.40·Futures + 0.30·PCR + 0.20·OI_bias + 0.10·IV_trend
Label:
    Bullish  if Score > +0.20  
    Bearish  if Score < −0.20  
    Neutral  otherwise

If futures data are missing (e.g., outside market hours),
only the options parts contribute.

------------------------------------------------------------------------------------
📂 OUTPUT FILES
------------------------------------------------------------------------------------
1. **fno_sentiment_summary.csv**  
   Per-symbol summary including:
   - `pcr`, `doi_net`, `iv_skew`, `basis`, `roll_spread`
   - `fut_regime`, `sentiment_score`, `sentiment_label`
   - Diagnostics: `has_options`, `has_futures`, `missing_fields`

2. **fno_sentiment_details.csv**  
   Strike-level view with OI, IV, and LTP for both calls and puts.

------------------------------------------------------------------------------------
📈 INTERPRETATION QUICK GUIDE
------------------------------------------------------------------------------------
| Futures + Options Mix | Likely Sentiment |
|------------------------|------------------|
| Long Buildup + High PCR | Strong Bullish |
| Short Buildup + Low PCR | Strong Bearish |
| Short Covering + Rising PCR | Bullish Reversal |
| Long Unwinding + Falling PCR | Bearish Reversal |

------------------------------------------------------------------------------------
🧩 TROUBLESHOOTING
------------------------------------------------------------------------------------
- `has_futures=False` → symbol not in F&O or endpoint gave no FUTSTK data.
- `missing_fields` → shows which keys were absent (`oi`, `prev_close`, etc.)
- Run during market hours for richer data.
- NSE may rate-limit; keep SYMBOLS list small (10–20 max) and respect pacing.

====================================================================================
"""

import os, time, math, datetime as dt
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
import requests
from dateutil import parser as dateparser
from pytz import timezone

# =========================
# CONFIG (EDIT HERE)
# =========================
SYMBOLS = ["RELIANCE", "HDFCBANK", "INFY", "ABB"]  # <— put your list here
IST = timezone("Asia/Kolkata")

REQUEST_TIMEOUT   = 12
MAX_RETRIES       = 5
RETRY_SLEEP_SEC   = 1.2
PACING_SLEEP_SEC  = 0.8
ATM_NEIGHBORS     = 2
SENTIMENT_THRESH  = 0.20

# Weights for composite sentiment
W_FUTURES = 0.40
W_PCR     = 0.30
W_OI_BIAS = 0.20
W_IV_TREND= 0.10

OUTDIR = os.path.join("outputs", dt.date.today().strftime("%Y-%m-%d"))
os.makedirs(OUTDIR, exist_ok=True)

# ---------------------------------------------------------------------------------
# HTTP SESSION (NSE)
# ---------------------------------------------------------------------------------
def make_session() -> requests.Session:
    s = requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.nseindia.com/",
        "Connection": "keep-alive",
    })
    # Warm cookies to avoid 401/403
    for _ in range(2):
        try:
            s.get("https://www.nseindia.com", timeout=REQUEST_TIMEOUT)
            break
        except requests.RequestException:
            time.sleep(0.6)
    return s

def _get_json(s: requests.Session, url: str, params: Optional[dict] = None) -> dict:
    for i in range(MAX_RETRIES):
        try:
            r = s.get(url, params=params, timeout=REQUEST_TIMEOUT)
            if r.status_code == 200:
                return r.json()
            if r.status_code in (401, 403):
                s.get("https://www.nseindia.com", timeout=REQUEST_TIMEOUT)
            time.sleep(RETRY_SLEEP_SEC * (i + 1))
        except requests.RequestException:
            time.sleep(RETRY_SLEEP_SEC * (i + 1))
    raise RuntimeError(f"Failed after {MAX_RETRIES} tries: {url}")

def fetch_option_chain_equity(s: requests.Session, symbol: str) -> dict:
    url = "https://www.nseindia.com/api/option-chain-equities"
    return _get_json(s, url, params={"symbol": symbol.upper()})

def fetch_futures_quote(s: requests.Session, symbol: str) -> dict:
    url = "https://www.nseindia.com/api/quote-derivative"
    return _get_json(s, url, params={"symbol": symbol.upper()})

# ---------------------------------------------------------------------------------
# UTILITIES
# ---------------------------------------------------------------------------------
def days_to(date_str: str, now_ist: dt.datetime) -> float:
    try:
        d = dateparser.parse(date_str)
        d_ist = IST.localize(dt.datetime(d.year, d.month, d.day, 15, 30))
        return max((d_ist - now_ist).total_seconds() / 86400.0, 0.0001)
    except Exception:
        return np.nan

def safe_tanh(x, scale=1.0):
    try:
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return np.nan
        return math.tanh(float(x) / float(scale if scale else 1.0))
    except Exception:
        return np.nan

# ---------------------------------------------------------------------------------
# OPTIONS METRICS
# ---------------------------------------------------------------------------------
def compute_options_metrics(oc_json: dict) -> dict:
    rec = oc_json.get("records", {})
    underlying = rec.get("underlyingValue", np.nan)
    data = oc_json.get("filtered", {}).get("data") or oc_json.get("records", {}).get("data") or []

    ce_oi = pe_oi = ce_oi_chg = pe_oi_chg = 0.0
    rows = []

    for row in data:
        strike = row.get("strikePrice")
        ce = row.get("CE"); pe = row.get("PE")

        if ce:
            ce_oi     += float(ce.get("openInterest", 0) or 0)
            ce_oi_chg += float(ce.get("changeinOpenInterest", 0) or 0)
        if pe:
            pe_oi     += float(pe.get("openInterest", 0) or 0)
            pe_oi_chg += float(pe.get("changeinOpenInterest", 0) or 0)

        rows.append({
            "strike": float(strike) if strike is not None else np.nan,
            "ce_oi": float(ce.get("openInterest", 0)) if ce else 0.0,
            "pe_oi": float(pe.get("openInterest", 0)) if pe else 0.0,
            "ce_chg_oi": float(ce.get("changeinOpenInterest", 0)) if ce else 0.0,
            "pe_chg_oi": float(pe.get("changeinOpenInterest", 0)) if pe else 0.0,
            "ce_iv": float(ce.get("impliedVolatility", np.nan)) if ce else np.nan,
            "pe_iv": float(pe.get("impliedVolatility", np.nan)) if pe else np.nan,
            "ce_ltp": float(ce.get("lastPrice", np.nan)) if ce else np.nan,
            "pe_ltp": float(pe.get("lastPrice", np.nan)) if pe else np.nan,
        })

    df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=[
        "strike","ce_oi","pe_oi","ce_chg_oi","pe_chg_oi","ce_iv","pe_iv","ce_ltp","pe_ltp"
    ])

    # PCR (guard zero)
    pcr = (pe_oi / ce_oi) if ce_oi not in (0, None) else np.nan
    pcr_score = safe_tanh((pcr - 1.0), scale=0.4) if not pd.isna(pcr) else np.nan

    # ΔOI bias (today’s tilt)
    doi_net = (pe_oi_chg - ce_oi_chg)
    oi_bias_score = safe_tanh(doi_net, scale=max(abs(doi_net), 1.0))

    # ATM IV from ±N strikes by index
    atm_ce_iv = atm_pe_iv = iv_skew = np.nan
    if not df.empty and not pd.isna(underlying):
        strikes = sorted(df["strike"].dropna().unique().tolist())
        if strikes:
            # find nearest strike index
            nearest_idx = int(np.argmin([abs(k - underlying) for k in strikes]))
            lo = max(nearest_idx - ATM_NEIGHBORS, 0)
            hi = min(nearest_idx + ATM_NEIGHBORS, len(strikes)-1)
            band_strikes = set(strikes[lo:hi+1])
            band = df[df["strike"].isin(band_strikes)]
            atm_ce_iv = band["ce_iv"].replace(0, np.nan).mean()
            atm_pe_iv = band["pe_iv"].replace(0, np.nan).mean()
            if not pd.isna(atm_ce_iv) and not pd.isna(atm_pe_iv):
                iv_skew = atm_pe_iv - atm_ce_iv

    return dict(
        has_options=True,
        underlying=underlying,
        pcr=pcr, pcr_score=pcr_score,
        doi_net=doi_net, oi_bias_score=oi_bias_score,
        atm_ce_iv=atm_ce_iv, atm_pe_iv=atm_pe_iv, iv_skew=iv_skew,
        oc_table=df
    )

# ---------------------------------------------------------------------------------
# FUTURES METRICS
# ---------------------------------------------------------------------------------
def parse_futures_from_derivative_quote(dq_json: dict) -> pd.DataFrame:
    futures = []
    def scan(node):
        if isinstance(node, dict):
            if node.get("instrumentType") in ("FUTSTK", "FUTIDX"):
                futures.append(node)
            for v in node.values():
                scan(v)
        elif isinstance(node, list):
            for x in node:
                scan(x)
    scan(dq_json)

    rows = []
    for f in futures:
        rows.append({
            "expiry": f.get("expiryDate") or f.get("expirydate") or f.get("expDate"),
            "ltp": float(f.get("lastPrice") or f.get("ltp") or f.get("closePrice") or np.nan),
            "oi": float(f.get("openInterest") or f.get("oi") or np.nan),
            "chg_oi": float(f.get("changeinOpenInterest") or f.get("changeInOpenInterest") or 0.0),
            "prev_close": float(f.get("previousClose") or f.get("prevClose") or np.nan),
        })

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df = df.dropna(subset=["expiry"]).copy()
    try:
        df["expiry_dt"] = df["expiry"].apply(lambda x: dateparser.parse(str(x)))
    except Exception:
        df["expiry_dt"] = pd.NaT
    return df.sort_values("expiry_dt").reset_index(drop=True)

def compute_futures_metrics(fut_df: pd.DataFrame, underlying: float, now_ist: dt.datetime) -> dict:
    if fut_df is None or fut_df.empty:
        return dict(
            has_futures=False, missing_fields="no_fut_rows",
            near_price=np.nan, next_price=np.nan, near_oi=np.nan, near_chg_oi=np.nan,
            basis=np.nan, basis_pct_ann=np.nan, roll_spread=np.nan,
            regime=None, fut_regime_score=np.nan, dte=np.nan, price_change=np.nan
        )

    near = fut_df.iloc[0]
    nxt = fut_df.iloc[1] if len(fut_df) > 1 else None

    # Validate fields
    missing = []
    near_price = float(near.get("ltp", np.nan))
    if pd.isna(near_price): missing.append("ltp")
    next_price = float(nxt.get("ltp", np.nan)) if nxt is not None else np.nan
    near_oi = float(near.get("oi", np.nan))
    if pd.isna(near_oi): missing.append("oi")
    near_chg_oi = float(near.get("chg_oi", np.nan))
    if pd.isna(near_chg_oi): missing.append("chg_oi")
    prev_close = float(near.get("prev_close", np.nan))
    if pd.isna(prev_close): missing.append("prev_close")
    near_exp = near.get("expiry")

    dte = days_to(near_exp, now_ist) if near_exp else np.nan
    if pd.isna(dte): missing.append("expiry")

    # Derived
    basis = near_price - float(underlying) if (not pd.isna(near_price) and not pd.isna(underlying)) else np.nan
    basis_pct_ann = ((basis / float(underlying)) * (365.0 / dte) * 100.0) if (not pd.isna(basis) and dte and dte > 0) else np.nan
    roll_spread = (next_price - near_price) if (not pd.isna(next_price) and not pd.isna(near_price)) else np.nan
    price_change = (near_price - prev_close) if (not pd.isna(near_price) and not pd.isna(prev_close)) else np.nan

    regime = None
    if not pd.isna(price_change) and not pd.isna(near_chg_oi):
        if price_change > 0 and near_chg_oi > 0:
            regime = "Long Buildup"
        elif price_change < 0 and near_chg_oi > 0:
            regime = "Short Buildup"
        elif price_change < 0 and near_chg_oi < 0:
            regime = "Long Unwinding"
        elif price_change > 0 and near_chg_oi < 0:
            regime = "Short Covering"

    fut_regime_score = {
        "Long Buildup": 1.0,
        "Short Buildup": -1.0,
        "Short Covering": 0.4,
        "Long Unwinding": -0.4
    }.get(regime, np.nan)

    return dict(
        has_futures=True if len(missing)==0 or len(missing)<5 else False,
        missing_fields=",".join(sorted(set(missing))) if missing else "",
        near_price=near_price, next_price=next_price,
        near_oi=near_oi, near_chg_oi=near_chg_oi,
        basis=basis, basis_pct_ann=basis_pct_ann,
        roll_spread=roll_spread, regime=regime,
        fut_regime_score=fut_regime_score, dte=dte, price_change=price_change
    )

# ---------------------------------------------------------------------------------
# SENTIMENT COMBINATION
# ---------------------------------------------------------------------------------
def combine_sentiment(fut_score, pcr_score, oi_bias_score, iv_skew) -> Tuple[float, str]:
    # IV trend proxy from skew: PE IV > CE IV → mild bullish
    if pd.isna(iv_skew):
        iv_trend_score = 0.0
    else:
        iv_trend_score = safe_tanh(iv_skew, scale=max(abs(iv_skew), 5.0))

    parts = []
    for val, w in [(fut_score, W_FUTURES), (pcr_score, W_PCR), (oi_bias_score, W_OI_BIAS), (iv_trend_score, W_IV_TREND)]:
        parts.append(0.0 if pd.isna(val) else w * float(val))

    score = float(np.nansum(parts))
    label = "Bullish" if score > SENTIMENT_THRESH else "Bearish" if score < -SENTIMENT_THRESH else "Neutral"
    return score, label

# ---------------------------------------------------------------------------------
# PROCESS ONE SYMBOL
# ---------------------------------------------------------------------------------
def process_symbol(s: requests.Session, symbol: str, now_ist: dt.datetime) -> dict:
    try:
        oc_json = fetch_option_chain_equity(s, symbol)
        oc_metrics = compute_options_metrics(oc_json)

        dq_json = fetch_futures_quote(s, symbol)
        fut_df = parse_futures_from_derivative_quote(dq_json)
        fut_metrics = compute_futures_metrics(fut_df, oc_metrics["underlying"], now_ist)

        score, label = combine_sentiment(
            fut_metrics.get("fut_regime_score", np.nan),
            oc_metrics.get("pcr_score", np.nan),
            oc_metrics.get("oi_bias_score", np.nan),
            oc_metrics.get("iv_skew", np.nan),
        )

        out = dict(
            symbol=symbol.upper(),
            time_ist=now_ist.strftime("%Y-%m-%d %H:%M:%S"),
            spot=oc_metrics.get("underlying", np.nan),

            has_options=oc_metrics.get("has_options", False),
            has_futures=fut_metrics.get("has_futures", False),
            missing_fields=fut_metrics.get("missing_fields", ""),

            # Options
            pcr=oc_metrics.get("pcr", np.nan),
            pcr_score=oc_metrics.get("pcr_score", np.nan),
            doi_net=oc_metrics.get("doi_net", np.nan),
            oi_bias_score=oc_metrics.get("oi_bias_score", np.nan),
            atm_ce_iv=oc_metrics.get("atm_ce_iv", np.nan),
            atm_pe_iv=oc_metrics.get("atm_pe_iv", np.nan),
            iv_skew=oc_metrics.get("iv_skew", np.nan),

            # Futures
            fut_near_price=fut_metrics.get("near_price", np.nan),
            fut_next_price=fut_metrics.get("next_price", np.nan),
            fut_near_oi=fut_metrics.get("near_oi", np.nan),
            fut_near_chg_oi=fut_metrics.get("near_chg_oi", np.nan),
            basis=fut_metrics.get("basis", np.nan),
            basis_pct_ann=fut_metrics.get("basis_pct_ann", np.nan),
            roll_spread=fut_metrics.get("roll_spread", np.nan),
            fut_regime=fut_metrics.get("regime", None),
            fut_regime_score=fut_metrics.get("fut_regime_score", np.nan),
            dte=fut_metrics.get("dte", np.nan),
            fut_price_change=fut_metrics.get("price_change", np.nan),

            # Final
            sentiment_score=score,
            sentiment_label=label,
        )

        out["_oc_table"] = oc_metrics.get("oc_table")
        out["_fut_table"] = fut_df
        return out

    except Exception as e:
        return dict(
            symbol=symbol.upper(),
            time_ist=now_ist.strftime("%Y-%m-%d %H:%M:%S"),
            error=str(e),
            has_options=False, has_futures=False, missing_fields="exception",
            sentiment_score=np.nan, sentiment_label="NA"
        )

# ---------------------------------------------------------------------------------
# MAIN
# ---------------------------------------------------------------------------------
def main():
    s = make_session()
    now_ist = dt.datetime.now(IST)
    results: List[dict] = []

    for sym in [x.strip().upper() for x in SYMBOLS if x.strip()]:
        print(f"[{now_ist.strftime('%H:%M:%S')}] Processing {sym} ...")
        res = process_symbol(s, sym, now_ist)
        results.append(res)
        time.sleep(PACING_SLEEP_SEC)

    # Summary CSV (add diagnostics columns up front)
    summary_cols = [
        "symbol","time_ist","spot",
        "has_options","has_futures","missing_fields",
        "pcr","pcr_score","doi_net","oi_bias_score","atm_ce_iv","atm_pe_iv","iv_skew",
        "fut_near_price","fut_next_price","fut_near_oi","fut_near_chg_oi",
        "basis","basis_pct_ann","roll_spread","fut_regime","fut_regime_score","dte","fut_price_change",
        "sentiment_score","sentiment_label","error"
    ]
    summary_df = pd.DataFrame([{k: r.get(k, np.nan) for k in summary_cols} for r in results])
    summary_path = os.path.join(OUTDIR, "fno_sentiment_summary.csv")
    summary_df.to_csv(summary_path, index=False)
    print("Wrote:", summary_path)

    # Details CSV (strike-level, if available)
    detail_rows = []
    for r in results:
        base = {k: r.get(k, np.nan) for k in summary_cols}
        oc_tbl = r.get("_oc_table")
        if isinstance(oc_tbl, pd.DataFrame) and not oc_tbl.empty:
            tmp = oc_tbl.copy()
            tmp["symbol"] = r["symbol"]; tmp["time_ist"] = r["time_ist"]
            for _, row in tmp.iterrows():
                detail_rows.append({
                    **base,
                    "strike": row.get("strike", np.nan),
                    "ce_oi": row.get("ce_oi", np.nan),
                    "pe_oi": row.get("pe_oi", np.nan),
                    "ce_chg_oi": row.get("ce_chg_oi", np.nan),
                    "pe_chg_oi": row.get("pe_chg_oi", np.nan),
                    "ce_iv": row.get("ce_iv", np.nan),
                    "pe_iv": row.get("pe_iv", np.nan),
                    "ce_ltp": row.get("ce_ltp", np.nan),
                    "pe_ltp": row.get("pe_ltp", np.nan),
                })
        else:
            detail_rows.append(base)

    details_df = pd.DataFrame(detail_rows)
    details_path = os.path.join(OUTDIR, "fno_sentiment_details.csv")
    details_df.to_csv(details_path, index=False)
    print("Wrote:", details_path)

if __name__ == "__main__":
    main()
